# Kapitola 7: Vytváření chatovacích aplikací
## Rychlý start s Github Models API

Tento zápisník je upravený z [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), kde najdete zápisníky využívající služby [Azure OpenAI](notebook-azure-openai.ipynb).


# Přehled  
„Velké jazykové modely jsou funkce, které převádějí text na text. Na základě zadaného textového řetězce se velký jazykový model snaží předpovědět, jaký text bude následovat“(1). Tento „rychlý start“ notebook uživatele seznámí se základními pojmy LLM, klíčovými požadavky na balíčky pro začátek práce s AML, stručným úvodem do návrhu promptů a několika krátkými příklady různých způsobů využití.


## Obsah  

[Přehled](../../../../07-building-chat-applications/python)  
[Jak používat OpenAI Service](../../../../07-building-chat-applications/python)  
[1. Vytvoření vaší OpenAI služby](../../../../07-building-chat-applications/python)  
[2. Instalace](../../../../07-building-chat-applications/python)    
[3. Přihlašovací údaje](../../../../07-building-chat-applications/python)  

[Příklady použití](../../../../07-building-chat-applications/python)    
[1. Shrnutí textu](../../../../07-building-chat-applications/python)  
[2. Klasifikace textu](../../../../07-building-chat-applications/python)  
[3. Generování nových názvů produktů](../../../../07-building-chat-applications/python)  
[4. Doladění klasifikátoru](../../../../07-building-chat-applications/python)  

[Odkazy](../../../../07-building-chat-applications/python)


### Vytvořte svůj první prompt  
Toto krátké cvičení vám poskytne základní úvod do zadávání promptů modelu v Github Models pro jednoduchý úkol „shrnutí“.


**Kroky**:  
1. Nainstalujte knihovnu `azure-ai-inference` do svého pythonového prostředí, pokud ji ještě nemáte.  
2. Načtěte standardní pomocné knihovny a nastavte přihlašovací údaje pro Github Models.  
3. Vyberte model pro svůj úkol  
4. Vytvořte jednoduchý prompt pro model  
5. Odešlete svůj požadavek na API modelu!


### 1. Nainstalujte `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Výběr správného modelu  
Modely GPT-3.5-turbo nebo GPT-4 dokážou rozumět a generovat přirozený jazyk.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Návrh promptů  

„Kouzlo velkých jazykových modelů spočívá v tom, že tím, jak jsou trénovány na minimalizaci chyby předpovědi na obrovském množství textu, nakonec se naučí pojmy užitečné pro tyto předpovědi. Například se naučí pojmy jako“(1):

* jak se správně píše
* jak funguje gramatika
* jak parafrázovat
* jak odpovídat na otázky
* jak vést konverzaci
* jak psát v různých jazycích
* jak programovat
* atd.

#### Jak ovládat velký jazykový model  
„Ze všech vstupů do velkého jazykového modelu je zdaleka nejvlivnější textový prompt.“(1)

Velké jazykové modely lze přimět k produkci výstupu několika způsoby:

Instrukce: Řekněte modelu, co chcete
Doplnění: Přimějte model, aby dokončil začátek toho, co chcete
Ukázka: Ukažte modelu, co chcete, a to buď:
Několika příklady v promptu
Stovkami nebo tisíci příkladů v trénovací datové sadě pro doladění



#### Existují tři základní zásady pro vytváření promptů:

**Ukažte a řekněte.** Jasně sdělte, co chcete, ať už pomocí instrukcí, příkladů, nebo jejich kombinací. Pokud chcete, aby model seřadil seznam položek podle abecedy nebo zařadil odstavec podle sentimentu, ukažte mu, že právě to chcete.

**Dávejte kvalitní data.** Pokud se snažíte vytvořit klasifikátor nebo chcete, aby model následoval určitý vzor, ujistěte se, že máte dostatek příkladů. Nezapomeňte si příklady zkontrolovat — model je obvykle dost chytrý na to, aby si všiml základních pravopisných chyb a odpověděl vám, ale také si může myslet, že to bylo záměrné, což může ovlivnit odpověď.

**Zkontrolujte svá nastavení.** Nastavení temperature a top_p ovlivňují, jak moc je model při generování odpovědi deterministický. Pokud po něm chcete odpověď, kde je jen jedna správná možnost, nastavte je níže. Pokud hledáte rozmanitější odpovědi, můžete je nastavit výše. Nejčastější chybou u těchto nastavení je předpokládat, že určují „chytrost“ nebo „kreativitu“ modelu.


Zdroj: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Shrnutí textu  
#### Výzva  
Shrňte text tak, že na konec textu přidáte 'tl;dr:'. Všimněte si, jak model dokáže vykonávat řadu úkolů bez dalších instrukcí. Můžete zkusit použít podrobnější výzvy než tl;dr, abyste upravili chování modelu a přizpůsobili si výsledné shrnutí(3).  

Nedávné studie ukázaly výrazné zlepšení v mnoha NLP úlohách a benchmarcích díky předtrénování na velkém korpusu textů, následovaném doladěním na konkrétní úlohu. I když je tato metoda obvykle nezávislá na konkrétní úloze, stále vyžaduje speciální doladěné datasety s tisíci nebo desítkami tisíc příkladů. Oproti tomu lidé zvládnou nový jazykový úkol často už po několika příkladech nebo jednoduchých instrukcích – což současné NLP systémy stále většinou nedokážou. Zde ukazujeme, že zvětšení jazykových modelů výrazně zlepšuje výkon bez ohledu na úlohu i při malém počtu příkladů, někdy dokonce dosahuje konkurenceschopnosti s předchozími špičkovými metodami doladění.  

Tl;dr


# Cvičení pro různé případy použití  
1. Shrnutí textu  
2. Klasifikace textu  
3. Generování nových názvů produktů


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Klasifikace textu  
#### Výzva  
Zařaďte položky do kategorií, které jsou zadány při vyhodnocení. V následujícím příkladu zadáváme v promptu jak kategorie, tak text, který má být klasifikován (*playground_reference).

Dotaz zákazníka: Dobrý den, na klávesnici mého notebooku se mi nedávno zlomila jedna klávesa a budu potřebovat náhradní:

Klasifikovaná kategorie:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Generujte nová jména produktů
#### Výzva
Vytvářejte názvy produktů na základě ukázkových slov. Do zadání zahrnujeme informace o produktu, pro který chceme názvy generovat. Také přidáváme podobný příklad, abychom ukázali vzor, který si přejeme obdržet. Nastavili jsme také vyšší hodnotu parametru temperature, abychom zvýšili náhodnost a získali inovativnější odpovědi.

Popis produktu: Domácí výrobník milkshaků  
Klíčová slova: rychlý, zdravý, kompaktní.  
Názvy produktů: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Popis produktu: Pár bot, které padnou na jakoukoliv velikost nohy.  
Klíčová slova: přizpůsobivý, padnoucí, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Reference  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Nejlepší postupy pro doladění GPT-3 pro klasifikaci textu](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Pro další pomoc  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Přispěvatelé
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho rodném jazyce. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné výklady vzniklé v důsledku použití tohoto překladu.
